COPYRIGHT © 2018 Kiran Arun <kironni@gmail.com>

### Setup

In [1]:
# setup
!rm -r Reinforcement_Learning-101-challenge
!git clone https://github.com/KiranArun/Reinforcement_Learning-101-challenge.git 1>/dev/null
!bash Reinforcement_Learning-101-challenge/scripts/setup.sh

rm: cannot remove 'Reinforcement_Learning-101-challenge': No such file or directory
Cloning into 'Reinforcement_Learning-101-challenge'...
remote: Counting objects: 95, done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 95 (delta 51), reused 60 (delta 24), pack-reused 0
Unpacking objects: 100% (95/95), done.
Extracting templates from packages: 100%


In [2]:
# DEFINITELY DO NOT EDIT THIS CELL

import os
# Start noVNC remote Xwindows in a browser with TurboVNC
!bash /content/Reinforcement_Learning-101-challenge/scripts/kill-novnc.sh >/dev/null
!sleep 2
# start novnc server and expose via ngrok NOT HTTPS
os.environ['PATH'] += "${PATH}:/opt/VirtualGL/bin:/opt/TurboVNC/bin"
# hack line below to get vnc to start as ipython subprocess
!timeout 5 /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup 2>/dev/null
get_ipython().system_raw('python /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup > /content/.vnc/stdout 2>&1 &')
# ngrok remote tunnel
get_ipython().system_raw('/content/ngrok http 5901 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print('XWindow Link:', json.load(sys.stdin)['tunnels'][0]['public_url'])"
# get Xwindows going (right click to get openbox menu)
get_ipython().system_raw('DISPLAY=:1 openbox &')
# one-time-password
!grep "one-time password:" /content/.vnc/stdout

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
XWindow Link: http://a22ec943.ngrok.io
Full control one-time password: 42590875


In [0]:
# DO NOT EDIT THIS CELL

# set up Xvfb and attach it's own vncserver and test with spinning gears
!pkill Xvfb   # clean-up
!pkill x11vnc # clean-up

get_ipython().system_raw('DISPLAY=:1 xterm &')
get_ipython().system_raw('/usr/bin/Xvfb :99 -screen 0 640x480x24 &')
get_ipython().system_raw('/usr/bin/x11vnc -rfbport 5902 -forever -display :99 &')
!sleep 2
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 glxgears &')

In [0]:
# open a terminal into the glxgears session using TightVNC
get_ipython().system_raw('DISPLAY=:99 xterm -e /bin/bash -l -c "pkill glxgears"')
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 xterm -geometry 105x35 &')

# RL Challenge

In [5]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import gym

import keras
import keras.layers as layers
from rl import agents,memory,policy

Using TensorFlow backend.


### Hyperparameters

In [0]:
# Hyperparams

# number of steps to keep in experience buffer
memory_limit = 400

# discount value
gamma = 0.99

# how much to update target graph
target_model_update = 1e-2

# learning rate
learning_rate = 1e-2

# number of steps to sample from buffer to train on
batch_size = 32

In [7]:
# DO NOT EDIT

# setting environment
env = gym.make('CartPole-v1')
num_actions = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### Neural Network

In [8]:
keras.backend.clear_session()

# create neural net
nn = keras.models.Sequential()

# keep this as input layer
nn.add(layers.Flatten(input_shape=(1,) + env.observation_space.shape))

# keep this as output layer
nn.add(layers.Dense(num_actions))

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### Optimizer

https://keras.io/optimizers/

In [0]:
# set optimizer
optimizer = keras.optimizers.SGD(lr=learning_rate)

### Memory

In [0]:
# DO NOT EDIT

# set experience buffer
memory = memory.SequentialMemory(limit=memory_limit,
                                    window_length=1)

### Policy

choose from: https://github.com/keras-rl/keras-rl/blob/master/rl/policy.py

[info on choosing](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf)

In [0]:
# set policy
policy = policy.BoltzmannQPolicy()

### Agent

choose from: https://github.com/keras-rl/keras-rl/tree/master/rl/agents

and use: https://github.com/keras-rl/keras-rl/tree/master/examples

In [0]:
# create agent
model = agents.dqn.DQNAgent(model=nn,
                            nb_actions=num_actions,
                            memory=memory,
                            gamma=gamma,
                            batch_size=batch_size,
                            nb_steps_warmup=150,
                            target_model_update=target_model_update,
                            policy=policy)

In [0]:
# DO NOT EDIT

# compile model
model.compile(optimizer,
            metrics=['mae'])

### Training

In [14]:
# DO NOT EDIT

# train
history = model.fit(env,
                  nb_steps=1000,
                  verbose=2)

Training for 1000 steps ...
   8/1000: episode: 1, duration: 0.258s, episode steps: 8, steps per second: 31, episode reward: 8.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.125 [0.000, 1.000], mean observation: 0.156 [-1.190, 2.016], loss: --, mean_absolute_error: --, mean_q: --
  18/1000: episode: 2, duration: 0.017s, episode steps: 10, steps per second: 586, episode reward: 10.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.900 [0.000, 1.000], mean observation: -0.140 [-2.765, 1.795], loss: --, mean_absolute_error: --, mean_q: --
  40/1000: episode: 3, duration: 0.038s, episode steps: 22, steps per second: 575, episode reward: 22.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.364 [0.000, 1.000], mean observation: 0.037 [-1.209, 1.984], loss: --, mean_absolute_error: --, mean_q: --
  55/1000: episode: 4, duration: 0.021s, episode steps: 15, steps per second: 700, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.400 [0.000, 1.000], mean 

 626/1000: episode: 37, duration: 0.216s, episode steps: 24, steps per second: 111, episode reward: 24.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.417 [0.000, 1.000], mean observation: 0.055 [-0.959, 1.492], loss: 0.332434, mean_absolute_error: 1.328621, mean_q: 2.049362
 638/1000: episode: 38, duration: 0.107s, episode steps: 12, steps per second: 112, episode reward: 12.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.667 [0.000, 1.000], mean observation: -0.100 [-1.524, 0.841], loss: 0.346361, mean_absolute_error: 1.362377, mean_q: 2.107700
 651/1000: episode: 39, duration: 0.124s, episode steps: 13, steps per second: 105, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.231 [0.000, 1.000], mean observation: 0.111 [-1.333, 2.253], loss: 0.345310, mean_absolute_error: 1.392566, mean_q: 2.159372
 667/1000: episode: 40, duration: 0.151s, episode steps: 16, steps per second: 106, episode reward: 16.000, mean reward: 1.000 [1.000, 1.000], mean ac

### Testing

In [15]:
# DO NOT EDIT

# test
hist = model.test(env, nb_episodes=50, visualize=False)
print('mean:', np.mean(hist.history['episode_reward']))

Testing for 50 episodes ...
Episode 1: reward: 10.000, steps: 10
Episode 2: reward: 10.000, steps: 10
Episode 3: reward: 10.000, steps: 10
Episode 4: reward: 8.000, steps: 8
Episode 5: reward: 10.000, steps: 10
Episode 6: reward: 10.000, steps: 10
Episode 7: reward: 10.000, steps: 10
Episode 8: reward: 9.000, steps: 9
Episode 9: reward: 10.000, steps: 10
Episode 10: reward: 9.000, steps: 9
Episode 11: reward: 10.000, steps: 10
Episode 12: reward: 11.000, steps: 11
Episode 13: reward: 9.000, steps: 9
Episode 14: reward: 10.000, steps: 10
Episode 15: reward: 10.000, steps: 10
Episode 16: reward: 9.000, steps: 9
Episode 17: reward: 10.000, steps: 10
Episode 18: reward: 9.000, steps: 9
Episode 19: reward: 10.000, steps: 10
Episode 20: reward: 9.000, steps: 9
Episode 21: reward: 9.000, steps: 9
Episode 22: reward: 10.000, steps: 10
Episode 23: reward: 10.000, steps: 10
Episode 24: reward: 10.000, steps: 10
Episode 25: reward: 8.000, steps: 8
Episode 26: reward: 10.000, steps: 10
Episode 27:

### Displaying test games

The below cell will save just the neural net with its weights.

To run the test with display:

1. Go to the XWindow link, and login if you haven't already
2. In the TightVNC terminal, type and run:
   
    `run <number_repeats>`
    - replace `<number_repeats>` with however many games you want it to play (or leave it blank to run once)


In [0]:
# save model to file
nn.save('/content/models/nn.hdf5', overwrite=True)